In [ ]:
from nbtemplate import display_header
display_header('PiSox.ipynb')

In [ ]:
from mayavi import mlab
mlab.init_notebook('x3d', 800, 500, local=False)

In [ ]:
import numpy as np

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, QTable
import marxs.visualization.mayavi
from marxs.source import PointSource, FixedPointing
from marxs.visualization.mayavi import plot_object, plot_rays
from marxs import simulator

import sys
sys.path.append('../')
from redsox.redsox import PerfectRedsox, xyz2zxy
from redsox.pisox import PerfectPisox, conf

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
instrum = PerfectPisox()
KeepProb = simulator.KeepCol('probability')
instrum.postprocess_steps.append(KeepProb)

my_source = PointSource(coords=SkyCoord(30., 30., unit='deg'), energy=0.3 * u.keV,
                        polarization=None,
                        geomarea=instrum.elements[0].area)
my_pointing = FixedPointing(coords=SkyCoord(30., 30., unit='deg'),
                            reference_transform=xyz2zxy)
expt = 100 * u.s
photons = my_source.generate_photons(expt)
photons = my_pointing(photons)

photons = instrum(photons)


## PiSoX: How does is look?
In this section, I show how the PiSoX setup looks. I start with an interactive 3D view, which can be zoomed and rotated with the mouse in all supported browsers, pressing "r" returns the view to the initial position. See [the X3DOM documentation](https://www.x3dom.org/documentation/interaction/) for a full list of supported mouse and keyboard commands. The ray-trace setup makes some simplifications. In particular, the mirror is not modeled in 3D, but approximated by a 2D lens. In the 3D view, the position of the mirror modules is just indicated schematically by a cylinder that has the same radius as the outermost mirror surface.  

This is a monochromatic simulation with photon energies of 0.3 keV. Only rays that are detected in the end are shown and rays are colored according to the grating diffraction order.

In [ ]:
fig = mlab.figure()
mlab.clf()

out = plot_object(instrum, viewer=fig)

pos = instrum.KeepPos.format_positions()
ind = (photons['probability'] > 1e-3) & (photons['CCD_ID'] >= 0)
out = marxs.visualization.mayavi.plot_rays(pos[ind, :, :], scalar=photons['order'][ind], viewer=fig)


# move camera programatically mlab.view()
fig

In [ ]:
ind = photons['shell'] > 0
plt.plot(pos[ind, 1, 0], pos[ind, 1, 1], '.')
plt.plot(pos[~ind, 1, 0], pos[~ind, 1, 1], '.')
plt.xlim(80, 200)
plt.ylabel('y [mm]')
plt.xlabel('x [mm]')

This plot shows the position of photons on the aperture. Blue photons hit an active mirror shell, while orange photons are outside the area covered by the mirrors or hit bulk Si that the shells are made from instead of a the reflective side.

The following image shows a 3D figure again, zooming in on the focal plane.

In [ ]:
fig = mlab.figure()
mlab.clf()

out = plot_object(simulator.Sequence(elements=instrum.elements[-3:]), viewer=fig)
fig

## How might an observation look like?

Since we have only a single polarimatery channel, we have to rotate the instrument on the sky to cover enough range in rotation angles to sample all possible polarization directions. In the next few plots, I show simulations with the spectrum and flux of Mk 421. I'm assuming the source is fully polarized because that way the effect of the rotation on the sky is the easiest to see. In reality, the polarization fraction is going to be $<1$ and might depend on energy. Of course, we can simulate those scenarios, too, but the point here is to show how the data works in principle, so we pick the easiest scenario.

We simulate two observations with different polarization angles on the sky, which are chosen such that the first angle gives the maximal signal and the second one the minimal signal on the polarization channel. In a real observation, that angle is not known a-priory and the instrument needs to be rotated continuously or observe at at least three angels to be able to uniquely derive the polarization fration and angle from the polarization channels alone. Using the zeoths order as well, observations at two angles are sufficient. Anyway, the point here is to show a range of possible outcomes, so picking the maximal and minimal signal possible is useful for demonstration purposes.

In [ ]:
def evtlist2_detimage(evt, bins=100, subplotkw={'figsize': (8,3)}):

    fig, axes = plt.subplots(ncols=2, **subplotkw)
    for i in [0, 1]:
        ind = evt['CCD_ID'] == i
        ccd = np.histogram2d(evt['detpix_x'][ind], evt['detpix_y'][ind], weights=evt['probability'][ind], bins=bins)
        im = axes[i].imshow(ccd[0].T, extent=(ccd[1][0], ccd[1][-1], ccd[2][0], ccd[2][-1]), origin='lower')
        axes[i].set_title(f'CCD {i}')
        cbar = fig.colorbar(im, ax=axes[i])
        cbar.set_label('cts / bin')
    return fig, axes

In [ ]:
from marxs.source.source import poisson_process
spectrum = Table.read('../inputdata/mk421_spec.txt', format='ascii.no_header',
                      names=['wave','fluxperwave'])
spectrum['wave'].unit = u.Angstrom
spectrum = QTable(spectrum)
spectrum['fluxperwave'].unit = 1 / u.cm**2 / u.s / u.Angstrom
spectrum['fluxperbin'] = spectrum['fluxperwave'] * np.hstack([0 * u.Angstrom, np.diff(spectrum['wave'])])
spectrum['energy'] = spectrum['wave'].to(u.keV, equivalencies=u.spectral())
spectrum.sort('energy')
spectrum['fluxdensity'] = spectrum['fluxperbin'] / np.hstack([0 * u.keV, np.diff(spectrum['energy'])])

spectrum = spectrum[(spectrum['wave'] > 25. * u.Angstrom) & (spectrum['wave'] < 60. * u.Angstrom)]
flux = np.sum(spectrum['fluxperbin'])
mk421coords = SkyCoord.from_name('Mk 421')
mk421 = PointSource(coords=mk421coords, energy=spectrum, flux=poisson_process(flux),
                        polarization=0 * u.degree,
                        geomarea=instrum.elements[0].area)

In [ ]:
plt.plot(spectrum['energy'], spectrum['fluxdensity'])
plt.title('Mk 421 input spectrum')
plt.xlabel('energy [keV]')
out = plt.ylabel('Flux [photons / s / cm$^{-2}$ / keV]')

In [ ]:
pointing1 = FixedPointing(coords=mk421coords, roll=0 * u.degree, reference_transform=xyz2zxy)
pointing2 = FixedPointing(coords=mk421coords, roll=.5 * np.pi * u.rad, reference_transform=xyz2zxy)

In [ ]:
photons = mk421.generate_photons(10 * u.ks)
p1 = pointing1(photons.copy())
p2 = pointing2(photons.copy())
len(photons)

In [ ]:
p1 = instrum(p1)
p2 = instrum(p2)

In [ ]:
fig, axes = evtlist2_detimage(p1)

In [ ]:
fig, axes = evtlist2_detimage(p2)

The two plots above show parts of the CCD0 (which detects the zeroth order) and CCD1 (which detects the polarization signal) for the two simulations at different polarization angles. CCD0 simply shows the image of a point source. CCD1 displays two strips of data, one containing the photons from the upper part of the CAT grating stair, the other one from the lower part.

Because the simulation tracks photons weights, the simulation can deliver non-integer counts. Note how the general pattern in the CCD1 images is the same, but the scales differ by about an order of magnitude in flux. On the other hand, the different rates on CCD 0 are compatible with Poisson statistics.

In [ ]:
# Need to baffle?

for i in [0, 1]:
    ind = p2['CCD_ID'] == i
    plt.hist((p2['energy'][ind]).to(u.Angstrom, equivalencies=u.spectral()).value, 
             weights=p2['probability'][ind], bins=np.arange(17, 60),
             label=f'angle 1, CCD {i}')
    
for i in [0, 1]:
    ind = p1['CCD_ID'] == i
    plt.hist((p1['energy'][ind]).to(u.Angstrom, equivalencies=u.spectral()).value, 
             weights=p1['probability'][ind], bins=np.arange(17, 60),
             label=f'angle 2, CCD {i}', histtype='step')
    
plt.legend()
ax = plt.gca()
ax.set_yscale('log')
ax.set_ylim(1e-2, None)
ax.set_xlabel('wavelength [$\AA$]')
out = ax.set_ylabel('flux [photons / bin]')

In [ ]:
p = p2
ind = p['CCD_ID'] == 1
plt.hist((p['energy'][ind]).to(u.Angstrom, equivalencies=u.spectral()).value, 
             weights=p['probability'][ind], bins=np.arange(17, 60),
             label=f'angle 1, CCD {i}')
p['probability'][ind].sum(), len(p)

This plot shows the observed spectrum in CCD0 and CCD1. Given the limited energy resolution of the CCDs itself, the observed data won't allow the same resolution in the zeroth order that we will achive in the first order. In the plot, the different effective areas in the zeroth and first order are clearly visible. Despite observing the same source, in the zeroth order, the flux decreases with increasing wavelength, while the spectrum in the other channel is almost flat.

## How important is sky background?

The simulation of the X-ray background from a model provided by Eric Miller. The model describes the Cosmic X-ray Background, composed of a soft, thermal Galactic foreground component and power-law component to account for
unresolved AGN. The magnitude of the latter is applicable for a 1 arcmin PSF and is scales appropriately below. This model comes from Bautz+2009 (PASJ, 61, 1117) analysis of Abell 1795. 

'phabs' can be changed depending on the source, but this model is probably only applicale for a typical high-latitude source, and could be off for the softest energies where (e.g.) the LHB pops in with lower absorption.

The bit about the Local Hot Bubble is important for energies below 0.25 keV or so, and the bit about being able to resolve out at least some point sources is important for the AGN component. 

The simulation below is run with the default parameters provided by Eric. The goal of the simulations is to determine how diffuse sky background influences the rate in the polarization channel. Since the signal there is dispersed source count rates can be very low, so background can potentially become important. On the other hand, the multi-layer mirror will only reflect photons coming in with a very specific combination of angle and energy, reducing the background significantly.

In [ ]:
from astropy.table import QTable
bkgspectrum = QTable.read('../inputdata/eric_bkg_model.tbl', format='ascii.no_header', 
                          names=['energy', 'fluxdensity'])

bkgspectrum['energy'] = bkgspectrum['energy'] * u.keV
bkgspectrum['fluxdensity'] = bkgspectrum['fluxdensity'] / u.cm**2 / u.s / u.keV
# Restrict to the energy range where I have parameters
bkgspectrum = bkgspectrum[(bkgspectrum['energy'] > 0.205 * u.keV) & (bkgspectrum['energy'] < 0.729 * u.keV)]

bkg_model_norm = (bkgspectrum['fluxdensity'][1:] * np.diff(bkgspectrum['energy'])).sum()

In [ ]:
from marxs.source import DiskSource

a_outer = 1 * u.degree
bkg_source = DiskSource(coords=SkyCoord(30., 30., unit='deg'), a_outer = a_outer,
                        polarization=None, energy=bkgspectrum, flux = bkg_model_norm * a_outer**2 / u.arcmin**2,
                        geomarea=instrum.elements[0].area)


In [ ]:
expt = 100 * u.ks
bkg = bkg_source.generate_photons(expt)
bkg = my_pointing(bkg)
print('Number of rays in simulated background:', len(bkg))

We set up a 100 ks simulation with abut 600,000 photons.

In [ ]:
bkg = instrum(bkg)

# In the current simulation, CCD 1 gets a lot of direct hits. Presumably, we'll baffle those out,
# so I just remove them here
ind = bkg['CCD_ID'] == i
ind2 = np.isfinite(bkg['mlwave_nominal'])

# presumably, any direct illumination will be baffled out
bkgbaffle = bkg[(bkg['CCD_ID']==0) | (ind & ind2)]

In [ ]:
out = plt.hist((bkg['energy']).to(u.Angstrom, equivalencies=u.spectral()).value, 
             bins=np.arange(17, 60))
plt.xlabel('wavelength [$\AA$]')
out = plt.ylabel('counts per bin')

The simulated spectrum has a broad energy distribution. At shorter wavelengths, it is dominated by strong oxygen emission lines, at longer wavelength, the spectrum is smoother.

In [ ]:
for i in [0, 1]:
    ind = bkgbaffle['CCD_ID'] == i
    plt.hist((bkgbaffle['energy'][ind]).to(u.Angstrom, equivalencies=u.spectral()).value, 
             weights=bkgbaffle['probability'][ind], bins=np.arange(17, 60),
             label=f'CCD {i}')
plt.legend()
ax = plt.gca()
ax.set_yscale('log')
ax.set_ylim(1e-3, 1e3)
ax.set_xlabel('wavelength [Ang]')
ax.set_ylabel('detected flux [cts / bin / 100 ks]')
ax.set_title('Background signal over entire detector')

Spectral distribution of detected background counts summed over the entire detector.

In [ ]:
fig, axes = evtlist2_detimage(bkg, bins=[1039//10, 2048//10])

Now we need an estimate of how many of those counts are in the extraction regions. To do that, I'll look at a simulated observatons from an on-axis point source and define the extraction regions as the region on the chip that gets signal there.

In [ ]:
ind = p1['CCD_ID'] == 1
p1bins = np.histogram2d(p1['detpix_x'][ind], p1['detpix_y'][ind], weights=p1['probability'][ind],
                        bins=25)

In [ ]:
out = plt.imshow(p1bins[0].T > 5, 
                 extent=(p1bins[1][0], p1bins[1][-1], p1bins[2][0], p1bins[2][-1]),
                 origin='lower')

In [ ]:
binx = p1bins[1][2] - p1bins[1][1]
biny = p1bins[2][2] - p1bins[2][1]

We'll use this pattern as the mask for extracting the background signal. It's a little rough around the edges and not 100% correct. For the real data analysis, we will define the extraction regions analytically. However, in the design phase the position and size of the detector might change, so it is useful to have a simple prgrammatic way to just select detector regions that contain the signal for an on-axis point source and use that region as the extraction region.

In [ ]:
ind = bkgbaffle['CCD_ID'] == 1
bkgbins = np.histogram2d(bkgbaffle['detpix_x'][ind], bkgbaffle['detpix_y'][ind],
                         weights=bkgbaffle['probability'][ind], bins=p1bins[1:])

In [ ]:
# Note: limit 1e-1 is chosen by hand for now
print('CCD1: extraction region size (pixels): {:6.0f} - total number of counts: {:6f}'.format(binx * biny * (p1bins[0].T > 1e-1).sum(),
              (bkgbins[0] * (p1bins[0].T > 5)).sum()))

In [ ]:
# What is that number of pixels in on-sky-region?
(313434 * (0.024 * u.mm)**2 * (u.rad/1.25/u.m)**2).to(u.arcmin**2)

In [ ]:
ind = p2['CCD_ID'] == 0
p2bins = np.histogram2d(p2['detpix_x'][ind], p2['detpix_y'][ind], weights=p2['probability'][ind],
                        bins=25)
binx = p2bins[1][2] - p2bins[1][1]
biny = p2bins[2][2] - p2bins[2][1]
binx * biny * (p2bins[0].T > 1e3).sum()

ind = bkgbaffle['CCD_ID'] == 0
bkgbins = np.histogram2d(bkgbaffle['detpix_x'][ind], bkgbaffle['detpix_y'][ind],
                         weights=bkgbaffle['probability'][ind], bins=p2bins[1:])

binx * biny * (p2bins[0].T > 200).sum(), (bkgbins[0] * (p2bins[0].T > 200)).sum()
print('CCD0: extraction region size (pixels): {:6.0f} - total number of counts: {:6f}'.format(
                binx * biny * (p2bins[0].T > 200).sum(),
              (bkgbins[0] * (p2bins[0].T > 200)).sum()))

### A modulation curve for one energy

In this section, I simulate a source that's 100% polarized. Then, I turn the pointing of the instrument on the sky in small steps so that we can build up the modulation curve. From this curve, the total modulation of the signal can be calcualted as $\frac{T-B}{T + B}$, where $T$ is the maximum of the curve and $B$ is the minimum.

In [ ]:
e = 0.3 * u.keV
n_photons = 2e4
ang = np.arange(0.,  2 * np.pi, .3) * u.rad
pol = np.zeros(len(ang))
for i, a in enumerate(ang):
    mysource = PointSource(coords=SkyCoord(30., 30., unit='deg'),
                            energy=e, polarization=a)
    photons = mysource.generate_photons(n_photons * u.s)
    photons = my_pointing(photons)
    photons = instrum(photons)
    pol[i] = photons['probability'][photons['CCD_ID'] == 1].sum()

In [ ]:
plt.plot(ang.to(u.deg), pol / pol.max())
plt.xlabel('Polarization angle [deg]')
plt.ylabel('Detected signal [arbitrary scaling]')
plt.ylim(0, None)

The plot shows a modulation curve. As expected, the curve is double humped with a periodicity of 180 deg. The modulation factor is close to 90%.

## Effective area and modulation factor

We can now run the calculation of the effective area and the modulation over a range of wavelengths.

In [ ]:
# Actually, we run this simulation in an external script 
# because of how long it takes to run and read in results from a file here
point_fixed = Table.read('../run_results/pisox/pi_aeff_mod.fits')

wave = np.arange(17., 61., 1.) * u.Angstrom

fig = plt.figure(figsize=(6,3))
ax1 = fig.add_subplot(121)
for i in [1, 0]:
    ax1.plot(wave, point_fixed[0]['Aeff'][:, i] * instrum.elements[0].area.to(u.cm**2), 
             label=['imaging', 'spectro-\npolarimetry'][i])
ax1.legend()
ax1.set_yscale('log')
ax1.set_xlabel('wavelength [$\AA$]')
ax1.set_ylabel('effective area [cm$^2$]')
ax1.set_xlim([25, 60])
ax1.set_ylim([.1, 17])
#ax1.xaxis.set_major_locator( MaxNLocator(nbins=5) )
#ax1.grid()
    
ax2 = fig.add_subplot(122)
ax2.plot(wave, point_fixed[0]['modulation'][:, 1])
ax2.set_xlabel('wavelength [$\AA$]')
ax2.set_ylabel('modulation factor')
ax2.set_xlim([25, 60])
ax2.set_ylim([.7, 1])
#ax2.xaxis.set_major_locator( MaxNLocator(nbins=5) )
#ax2.grid()
    
fig.subplots_adjust(wspace=0.4)
fig.savefig('../pisoxplots/aeff.pdf', bbox_inches='tight')
fig.savefig('../pisoxplots/aeff.png', bbox_inches='tight')

*left:* Effective area. CCD1 detects the polarized signal, CCD 0 the zeroth order. Because CCD 1 observed the signal of the multi-layer mirror, the effective area is much lower than in the CCD 0. 
*right:* modulation factor.